## Forecasting and trading cryptocurrencies with machine learning under changing market conditions
https://jfin-swufe.springeropen.com/articles/10.1186/s40854-020-00217-x

In [ ]:
#Bitcoin, ethereum, litecoin
